# Import

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

import time
import re

from bs4 import BeautifulSoup

data_base_path = "C:\Potenup\Drug-Detection-Chatbot\data\\"

original_file_name = "단일 경구약제_5,000종 리스트.xlsx"
file_name = "single_drug_list_5000.csv"

In [3]:
# 보통 현재 작업 폴더(CWD)가 src 이므로 부모(프로젝트 루트)를 sys.path에 추가
import sys, os
from pathlib import Path

ROOT = Path.cwd().parent
if (ROOT / "utils").exists():
    sys.path.insert(0, str(ROOT))
else:
    # 혹시 구조가 더 깊을 때 대비
    ROOT2 = ROOT.parent
    if (ROOT2 / "utils").exists():
        sys.path.insert(0, str(ROOT2))

# 약 리스트 데이터 불러오기

In [ ]:
df_1 = pd.read_excel(data_base_path + original_file_name, sheet_name=0)

In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   C-
Code       1000 non-null   object 
 1   구분            1000 non-null   object 
 2   제품명           1000 non-null   object 
 3   복지부 
분류코드     1000 non-null   float64
 4   Unnamed: 4    0 non-null      float64
 5   Unnamed: 5    0 non-null      float64
 6   Unnamed: 6    0 non-null      float64
 7   Unnamed: 7    0 non-null      float64
 8   Unnamed: 8    0 non-null      float64
 9   Conncet
Code  5000 non-null   object 
 10  구분.1          5000 non-null   object 
 11  제품명.1         5000 non-null   object 
 12  복지부
분류코드      5000 non-null   int64  
dtypes: float64(6), int64(1), object(6)
memory usage: 507.9+ KB


In [ ]:
df_2 = pd.read_excel(data_base_path + original_file_name, sheet_name=1)

In [6]:
df_2.head()

,C-\nCode,구분,제품명,복지부 \n분류코드,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Conncet\nCode,구분.1,제품명.1,복지부\n분류코드
0,K-000027,일반의약품,아네모정 20T,234.0,NaN,NaN,NaN,NaN,NaN,K-000032,전문의약품,다이크로짇정 25MG (유한),213
1,K-000030,전문의약품,테라싸이클린캅셀 250MG (종근당),615.0,NaN,NaN,NaN,NaN,NaN,K-000034,일반의약품,페니라민정,141
2,K-000041,일반의약품,삐콤정 (유한),316.0,NaN,NaN,NaN,NaN,NaN,K-000040,전문의약품,유한짓정 (유한),622
3,K-000042,일반의약품,푸라콩정,141.0,NaN,NaN,NaN,NaN,NaN,K-000043,전문의약품,디고신정 0.25MG(HK이노엔),211
4,K-000065,전문의약품,크로세린캅셀 250MG (동아),613.0,NaN,NaN,NaN,NaN,NaN,K-000117,일반의약품,액티피드정 (삼일),141


In [7]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   C-
Code       3999 non-null   object 
 1   구분            3999 non-null   object 
 2   제품명           3999 non-null   object 
 3   복지부 
분류코드     3999 non-null   float64
 4   Unnamed: 4    0 non-null      float64
 5   Unnamed: 5    0 non-null      float64
 6   Unnamed: 6    0 non-null      float64
 7   Unnamed: 7    0 non-null      float64
 8   Unnamed: 8    0 non-null      float64
 9   Conncet
Code  5000 non-null   object 
 10  구분.1          5000 non-null   object 
 11  제품명.1         5000 non-null   object 
 12  복지부
분류코드      5000 non-null   int64  
dtypes: float64(6), int64(1), object(6)
memory usage: 507.9+ KB


# 약 리스트 데이터 전처리

## 결측치 제거

In [ ]:
# 필요 없는 칼럼 제거
df_1 = df_1.drop(["Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Conncet\nCode", "구분.1", "제품명.1", "복지부\n분류코드"], axis=1)

In [9]:
df_1.head()

,C-\nCode,구분,제품명,복지부 \n분류코드
0,K-000059,일반의약품,게루삼정 200T,234.0
1,K-000069,일반의약품,지엘타이밍정 12T,115.0
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0
3,K-000112,일반의약품,아로나민골드,316.0
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0


In [10]:
df_1.shape

(5000, 4)

In [11]:
df_1 = df_1.dropna()

In [12]:
df_1.shape

(1000, 4)

In [13]:
df_2 = df_2.drop(["Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Conncet\nCode", "구분.1", "제품명.1", "복지부\n분류코드"], axis=1)

In [14]:
df_2.shape

(5000, 4)

In [15]:
df_2 = df_2.dropna()

In [16]:
df_2.shape

(3999, 4)

## 병합

In [ ]:
# sheet1 과 sheet2 를 병합
original_df = pd.concat([df_1, df_2], axis=0)

In [18]:
original_df

,C-\nCode,구분,제품명,복지부 \n분류코드
0,K-000059,일반의약품,게루삼정 200T,234.0
1,K-000069,일반의약품,지엘타이밍정 12T,115.0
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0
3,K-000112,일반의약품,아로나민골드,316.0
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0
...,...,...,...,...
3994,K-051831,일반의약품,위엔싹에프정 10T,232.0
3995,K-052494,일반의약품,뉴락스에프정 10T,238.0
3996,K-052505,전문의약품,휴메트린정0.125mg,252.0
3997,K-053315,전문의약품,레코미드서방정 150MG(유한),232.0


In [ ]:
original_df.to_csv(data_base_path + file_name, index=False)

## 제품명 정제 (약명 / 용량)

In [4]:
df = pd.read_csv(data_base_path + file_name)

In [7]:
df.head()

,C-Code,구분,제품명,복지부분류코드,volume,name
0,K-000059,일반의약품,게루삼정 200T,234.0,200T,게루삼정
1,K-000069,일반의약품,지엘타이밍정 12T,115.0,12T,지엘타이밍정
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0,400mg,마이암부톨제피정
3,K-000112,일반의약품,아로나민골드,316.0,NaN,아로나민골드
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0,480MG,셉트린정


In [32]:
# 제품명에서 약의 이름만 정제
def cleaned_name(name):
    s = re.sub(r"\([^)]*\)|\[.*?\]", "", name).strip()
    # s = re.sub(r"\([^)]*\)", "", name)
    s = re.sub(r"\s*\d+\s*[A-Za-z가-힣]*", "", s)
    s = re.sub(r"[-+*/.,:;_\s\(\)\[\]]", "", s)
    s = re.sub(r"[^A-Za-z가-힣]", "", s)
    s = s.strip()

    return s

In [33]:
# 제품명에서 약의 용량만 정제
def find_volume(name):
    pattern = re.compile(r"\d+\s*[A-Za-z가-힣]+")
    match = pattern.search(name)
    if match:
        return match.group()
    else:
        return ''

In [34]:
cleaned_name_list = []
volume_list = []
for i, data in df.iterrows():
    code, _, name, _, _, _ = data
    cleaned_name_list.append(cleaned_name(name))
    volume_list.append(find_volume(name))
    print("정제 : ", cleaned_name(name), ", 용량 : ", find_volume(name))

정제 :  게루삼정 , 용량 :  200T
정제 :  지엘타이밍정 , 용량 :  12T
정제 :  마이암부톨제피정 , 용량 :  400mg
정제 :  아로나민골드 , 용량 :  
정제 :  셉트린정 , 용량 :  480MG
정제 :  자이로릭정 , 용량 :  100MG
정제 :  다오닐정 , 용량 :  5mg
정제 :  보나링에이정 , 용량 :  
정제 :  에나폰정 , 용량 :  10mg
정제 :  키모랄에스정 , 용량 :  
정제 :  후릭스정 , 용량 :  40mg
정제 :  리단정 , 용량 :  300mg
정제 :  마그밀정 , 용량 :  500mg
정제 :  낙센정 , 용량 :  250mg
정제 :  구주스피로닥톤정 , 용량 :  25mg
정제 :  페리나정 , 용량 :  25mg
정제 :  로프민캡슐 , 용량 :  10C
정제 :  세가톤트로키정 , 용량 :  12T
정제 :  후시딘정 , 용량 :  250mg
정제 :  네오메디코푸정 , 용량 :  
정제 :  피라진아미드정 , 용량 :  500MG
정제 :  노이시린에이정 , 용량 :  500T
정제 :  쎄닐톤정 , 용량 :  
정제 :  메티마졸정 , 용량 :  
정제 :  게보린 , 용량 :  10T
정제 :  싸이메트정 , 용량 :  300mg
정제 :  슈다페드정 , 용량 :  
정제 :  베라드정 , 용량 :  10T
정제 :  프리미돈정 , 용량 :  250MG
정제 :  트렌탈서방정 , 용량 :  400MG
정제 :  뮤코론캅셀 , 용량 :  
정제 :  베노라제정 , 용량 :  10T
정제 :  우루사정 , 용량 :  100MG
정제 :  베타록정 , 용량 :  100MG
정제 :  스파부틴정 , 용량 :  30T
정제 :  리포덱스정 , 용량 :  450MG
정제 :  타라부틴정 , 용량 :  100mg
정제 :  디스토시드정 , 용량 :  600MG
정제 :  리팜핀정 , 용량 :  600mg
정제 :  부루펜정 , 용량 :  400mg
정제 :  파자임 , 용량 :  95 이

In [35]:
# if 'name' not in df.columns:
df['name'] = cleaned_name_list
if 'volume' not in df.columns:
    df['volume'] = volume_list

In [36]:
df.head()

,C-Code,구분,제품명,복지부분류코드,volume,name
0,K-000059,일반의약품,게루삼정 200T,234.0,200T,게루삼정
1,K-000069,일반의약품,지엘타이밍정 12T,115.0,12T,지엘타이밍정
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0,400mg,마이암부톨제피정
3,K-000112,일반의약품,아로나민골드,316.0,NaN,아로나민골드
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0,480MG,셉트린정


In [26]:
df['name'].value_counts()

name
씬지로이드정     7
씬지록신정      7
트리레보정      6
자디앙듀오정     6
아모잘탄큐정     6
          ..
파모타제정      1
위엔싹에프정     1
뉴락스에프정     1
휴메트린정      1
레코미드서방정    1
Name: count, Length: 4288, dtype: int64

In [37]:
df.to_csv(data_base_path + file_name, index=False)

# 크롬 웹 드라이버 생성

In [3]:
from utils.driver import make_driver, polite_sleep

In [4]:
main_url = "https://www.health.kr/main.asp"

In [5]:
# 드라이버 생성
driver = make_driver()

time.sleep(2)

driver.get(main_url)

# 약 검색 HTML 크롤링

약에 대한 정보를 검색 후 HTML 

## 검색 후 html 크롤링

In [ ]:
from selenium.common.exceptions import TimeoutException

def search_keyword_html(driver, keyword):
    try:
        input_box_css = "#search_word"
        input_box = driver.find_element(By.CSS_SELECTOR, input_box_css)
        input_box.clear()
        input_box.send_keys(keyword)
        input_box.send_keys(Keys.ENTER)

        polite_sleep(3)

        # try:
        #     popup = WebDriverWait(driver, 3).until(
        #         EC.presence_of_element_located((By.CSS_SELECTOR, ".popup-close"))
        #     )
        #     popup.click()
        #     polite_sleep(2)
        # except TimeoutException:
        #     pass
        
        html = driver.page_source
        return html
    except:
        return ""

In [39]:
df = pd.read_csv(data_base_path + file_name)

In [40]:
keyword_list = df['name'].unique().tolist()
html_list = [''] * len(keyword_list)

In [68]:
import os
if os.path.exists(data_base_path + "drug_search_html.csv"):
    keyword_html_df = pd.read_csv(data_base_path + "drug_search_html.csv")
else :
    keyword_html_df = pd.DataFrame({
    "keyword": keyword_list,
    "html" : html_list
})

In [74]:
keyword_html_df.head()

,keyword,html
0,게루삼정,
1,지엘타이밍정,
2,마이암부톨제피정,
3,아로나민골드,
4,셉트린정,


In [76]:
start = 0
end = 1000
for i, keyword in enumerate(keyword_list):
    if i < start or i > end:
        continue
    if pd.isna(keyword_html_df.iloc[i]['html']) or keyword_html_df.iloc[i]['html'] == "":
        html = search_keyword_html(driver, keyword)
        keyword_html_df.loc[i, 'html'] = html
        # html_list.append(html)
        keyword_html_df.to_csv(data_base_path + "drug_search_html.csv", index=False)
        print(f"{i + 1}/{len(keyword_list)} {keyword}")

1/4288 게루삼정
2/4288 지엘타이밍정
3/4288 마이암부톨제피정
4/4288 아로나민골드
5/4288 셉트린정
6/4288 자이로릭정
7/4288 다오닐정
8/4288 보나링에이정
9/4288 에나폰정
10/4288 키모랄에스정
11/4288 후릭스정
12/4288 리단정
13/4288 마그밀정
14/4288 낙센정
15/4288 구주스피로닥톤정
16/4288 페리나정
17/4288 로프민캡슐
18/4288 세가톤트로키정
19/4288 후시딘정
20/4288 네오메디코푸정
21/4288 피라진아미드정
22/4288 노이시린에이정
23/4288 쎄닐톤정
24/4288 메티마졸정
25/4288 게보린
26/4288 싸이메트정
27/4288 슈다페드정
28/4288 베라드정
29/4288 프리미돈정
30/4288 트렌탈서방정
31/4288 뮤코론캅셀
32/4288 베노라제정
33/4288 우루사정
34/4288 베타록정
35/4288 스파부틴정
36/4288 리포덱스정
37/4288 타라부틴정
38/4288 디스토시드정
39/4288 리팜핀정
40/4288 부루펜정
41/4288 파자임
42/4288 스코펜정
43/4288 크로낙정
44/4288 트라우밀정제
45/4288 소말겐정
46/4288 명인할로페리돌정
47/4288 명인벤즈트로핀정
48/4288 클로르프로마진정
49/4288 에세푸릴캡슐
50/4288 헤르벤서방정
51/4288 칸택스정
52/4288 켈론정
53/4288 디크로닉정
54/4288 와르파린나트륨정
55/4288 트리메부틴말레산염정
56/4288 베아제정
57/4288 알마겔정
58/4288 부스파정
59/4288 카디아젬정
60/4288 피케이멜즈정
61/4288 자비론정
62/4288 지엘소브레롤캡슐
63/4288 젠텔정
64/4288 에나프린정
65/4288 젤콤정
66/4288 베스타제포르테정
67/4288 케파신캅셀
68/4288 이부펜정
69/4288 크록신정
70/4288 트리마정
71/4288 사이톱신정
72/4288

In [ ]:
keyword_html_df.to_csv(data_base_path + "drug_search_html.csv", index=False)

In [77]:
len(keyword_html_df.loc[keyword_html_df['html'] != ''])

2362

## 병합

In [110]:
df_0 = pd.read_csv(data_base_path + "drug_search_html_0.csv")
df_1 = pd.read_csv(data_base_path + "drug_search_html_1001.csv")
df_2 = pd.read_csv(data_base_path + "drug_search_html_2001.csv")
df_3 = pd.read_csv(data_base_path + "drug_search_html_3001.csv")

In [111]:
df_merged = df_0.copy()
df_merged['html'] = df_merged['html'].fillna(
    df_merged['keyword'].map(df_1.set_index('keyword')['html'])
)
df_merged['html'] = df_merged['html'].fillna(
    df_merged['keyword'].map(df_2.set_index('keyword')['html'])
)
df_merged['html'] = df_merged['html'].fillna(
    df_merged['keyword'].map(df_3.set_index('keyword')['html'])
)

In [112]:
df_merged.isna().sum()

keyword      0
html       126
dtype: int64

In [113]:
df_merged.to_csv(data_base_path + "drug_search_html.csv", index=False)

## 약 리스트 HTML 정제

In [114]:
keyword_html_df = pd.read_csv(data_base_path + "drug_search_html.csv")

In [115]:
def find_keyword_code(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    tds_with_onclick = [
    td for td in soup.find_all("td", class_="txtL") if td.has_attr("onclick")
    ]

    code_list = []
    pattern = re.compile(r"\('([^']+)'\)")  # 작은따옴표 안의 문자열만 추출
    for td in tds_with_onclick:
        onclick_val = td["onclick"]
        match = pattern.search(onclick_val)
        if match:
            param = match.group(1)
            print(td.text, param)
            code_list.append({
                'name' : td.text,
                'code' : param
            })
    
    return code_list

In [143]:
code_list = []
for i, row in keyword_html_df.iterrows():
    print(f"{i}: ", end="")
    if pd.isna(row['html']) or row['html'] == "":
        continue
    code_list.extend(find_keyword_code(row['html']))

0: 게루삼정 A11ABBBBB2281
1: 지엘타이밍정 A11APPPPP1610
2: 마이암부톨제피정400mg A11A0450A0151
3: 아로나민골드정 A11A0340A0195
아로나민골드프리미엄정 2021030500011
4: 셉트린정 A11A0500A0065
5: 자이로릭정 A11A0500A0069
6: 다오닐정 A11ABBBBB0953
7: 보나링에이정 A11A0950A0043
8: 에나폰정10mg A11A0970A0064
에나폰정5mg 2013103000008
9: 키모랄에스정 A11A0770B0015
10: 후릭스정 A11A0950A0068
11: 리단정 A11A1310A0026
12: 마그밀정 A11ABBBBB1168
13: 낙센정 A11A0120A0286
14: 구주스피로닥톤정 A11A2780A0045
15: 페리나정 A11A1270A0036
셀페리나정10mg 2017081600006
셀페리나정5mg 2017081600007
16: 로프민캡슐 A11A1660A0057
17: 18: 후시딘정 A11A0570A0188
19: 네오메디코푸정 A11ABBBBB0607
20: 유한피라진아미드정250mg A11A0450A0294
유한피라진아미드정500mg A11A0450A0295
코러스피라진아미드정500mg(수출용) A11A5060A0010
코러스피라진아미드정(수출용) A11A5060A0017
21: 일양노이시린에이정 A11A0950A0131
22: 쎄닐톤정 A11A1110A0058
23: 부광메티마졸정 A11A1310A0130
한국유나이트메티마졸정(수출용) A11AFFFFF0079
부광메티마졸정2.5mg 2021012100004
24: 게보린정 A11A1270A0060
소아용게보린아이츄정(수출용) A11AMMMMM1151
게보린에스정(수출용) 2011122300007
게보린에프정(수출용) 2012011100004
게보린소프트연질캡슐 2019121000003
게보린쿨다운정 2020100700002
게보린릴랙스연질캡슐 2020102900010
게보린브이정

In [144]:
drug_code_df = pd.DataFrame(code_list)

In [145]:
drug_code_df.to_csv(data_base_path + "drug_code.csv", index=False)

# 약 세부정보 HTML 크롤링

In [6]:
drug_code_df = pd.read_csv(data_base_path + "drug_code.csv")

In [7]:
# 초기화
if 'complete' not in drug_code_df.columns:
    drug_code_df['complete'] = [False] * len(drug_code_df)

In [8]:
drug_code_df

,name,code,complete
0,게루삼정,A11ABBBBB2281,True
1,지엘타이밍정,A11APPPPP1610,True
2,마이암부톨제피정400mg,A11A0450A0151,True
3,아로나민골드정,A11A0340A0195,True
4,아로나민골드프리미엄정,2021030500011,True
...,...,...,...
6058,파모타제정10mg,2020041300004,False
6059,위엔싹에프정,2020051800011,False
6060,뉴락스에프정,2020070800006,False
6061,휴메트린정,2020071000008,False


In [9]:
result_drug_base_url = "https://www.health.kr/searchDrug/result_drug.asp?drug_cd="
result_take_base_url = "https://www.health.kr/searchDrug/result_take.asp?drug_cd="
result_sunb_base_url = "https://www.health.kr/searchDrug/result_sunb.asp?drug_cd="
result_interaction_base_url = "https://www.health.kr/searchDrug/result_interaction.asp?drug_cd="

In [10]:
import os
if os.path.exists(data_base_path + "drug_detail_html.csv"):
    drug_detail_html = pd.read_csv(data_base_path + "drug_detail_html.csv")
else :
    drug_detail_data = {
        'code' : drug_code_df['code'],
        'detail_html' : [""] * len(drug_code_df),
    }
    drug_detail_html = pd.DataFrame(drug_detail_data)
    drug_detail_html.to_csv(data_base_path, "drug_detail_html.csv", index=False)

In [11]:
drug_detail_html

,code,detail_html
0,A11ABBBBB2281,"<html lang=""ko""><head>\n<meta http-equiv=""X-UA..."
1,A11APPPPP1610,"<html lang=""ko""><head>\n<meta http-equiv=""X-UA..."
2,A11A0450A0151,"<html lang=""ko""><head>\n<meta http-equiv=""X-UA..."
3,A11A0340A0195,"<html lang=""ko""><head>\n<meta http-equiv=""X-UA..."
4,2021030500011,"<html lang=""ko""><head>\n<meta http-equiv=""X-UA..."
...,...,...
6058,2020041300004,NaN
6059,2020051800011,NaN
6060,2020070800006,NaN
6061,2020071000008,NaN


In [12]:
def get_drug_detail_html(driver, code):
    try:
        driver.get(result_drug_base_url + code)
        polite_sleep(2)
        return driver.page_source
    except:
        return ""

In [ ]:
start = 0
end = 3000
for i, row in drug_code_df.iterrows():
    if i < start or i > end:
        continue
    if pd.isna(drug_detail_html.loc[i, 'detail_html']):
        code = row['code']
        html = get_drug_detail_html(driver, code)

        drug_detail_html.loc[i] = [code, html]
        drug_code_df.at[i, 'complete'] = True

        if i % 5 == 0 or i == len(drug_code_df) - 1:
            drug_detail_html.to_csv(os.path.join(data_base_path, "drug_detail_html.csv"), index=False)
            drug_code_df.to_csv(os.path.join(data_base_path, "drug_code.csv"), index=False)
            print("Data saved.")

        print(f"{i}/{len(drug_code_df)} {code}")

Data saved.
300/6063 A11ABBBBB1538
301/6063 A11ABBBBB1539
302/6063 A11ABBBBB1593
303/6063 A11AGGGGG5333
304/6063 A11ABBBBB1722
Data saved.
305/6063 A11ALLLLL0136
306/6063 A11ABBBBB0292
307/6063 A11ADDDDD0063
308/6063 2013052000002
309/6063 A11AHHHHH0014
Data saved.
310/6063 A11AKP08H0034
311/6063 A11AOOOOO0804
312/6063 A11AGGGGA5803
313/6063 A11AJJJJJ0729
314/6063 A11AHHHHH0285
Data saved.
315/6063 A11AHHHHH0345
316/6063 A11ABBBBB1548
317/6063 A11ALLLLL0175
318/6063 A11AMMMMM3078
319/6063 2023120800005
Data saved.
320/6063 A11A3060A0221
321/6063 A11AGGGGA1783
322/6063 A11AGGGGA0252
323/6063 A11AJJJJJ0013
324/6063 A11AGGGGA4024
Data saved.
325/6063 A11AGGGGA5213
326/6063 A11A2070B0127
327/6063 A11AGGGGA3132
328/6063 A11AKKKKK0014
329/6063 A11AGGGGA5893
Data saved.
330/6063 A11AMMMMM2584
331/6063 A11AHHHHH0352
332/6063 A11AFFFFF0255
333/6063 A11AKP08C0213
334/6063 A11AKP08G3004
Data saved.
335/6063 2012080100009
336/6063 2013052300002
337/6063 2016012800007
338/6063 A11A0840A0196
339/606

## 병합

In [33]:
df_0 = pd.read_csv(data_base_path + "drug_detail_html_0.csv")
df_1 = pd.read_csv(data_base_path + "drug_detail_html_3000.csv")

In [34]:
df_0 = df_0.dropna()
df_1 = df_1.dropna()

In [35]:
drug_detail_html = pd.concat([df_0, df_1], ignore_index=True)

## 약 세부 정보 정제

In [ ]:
drug_detail_html = pd.read_csv(data_base_path + "drug_detail_html.csv")

In [36]:
def get_drug_detail(html):
    soup = BeautifulSoup(html, "html.parser")

    # 1. 제품명
    result_drug_name = soup.select_one("#result_drug_name")
    if result_drug_name:
        result_drug_name = result_drug_name.get_text(strip=True)

    # 2. 성분/함량 (li > a)
    ingr_mg = [a.get_text(strip=True) for a in soup.select("#ingr_mg li a")]

    # 3. 첨가물
    additives_raw = soup.select_one("#additives").get_text("\n", strip=True)
    additives = [a.strip('" ') for a in additives_raw.split("\n") if a.strip()]

    # 4. 제형
    drug_form = soup.select_one("#drug_form")
    if drug_form:
        drug_form = drug_form.get_text(strip=True)

    # 5. 성상
    charact = soup.select_one("#charact")
    if charact:
        charact = charact.get_text(strip=True)

    # 6. KPIC/ATC 코드 (li > a)
    kpic_atc = [a.get_text(strip=True) for a in soup.select("#kpic_atc li a")]

    # 7. 전문/일반 구분
    stmt = soup.select_one("#stmt")
    if stmt:
        stmt = stmt.get_text(strip=True)

    # 8. 효능
    effect = soup.select_one("#effect")
    if effect:
        effect = effect.get_text(" ", strip=True)

    # 9. 용법/용량
    dosage = soup.select_one("#dosage")
    if dosage:
        dosage = dosage.get_text(" ", strip=True)

    # 10. 주의사항
    caution = soup.select_one("#caution")
    if caution:
        caution = caution.get_text(" ", strip=True)

    result = {
        "제품명": result_drug_name,
        "성분/함량": ingr_mg,
        "첨가물": additives,
        "제형": drug_form,
        "성상": charact,
        "KPIC/ATC": kpic_atc,
        "구분": stmt,
        "효능": effect,
        "용법": dosage,
        "주의사항": caution,
    }
    
    return result


In [37]:
detail_data = []
for i, row in drug_detail_html.iterrows():
    detail_data.append(get_drug_detail(row['detail_html']))
    print(f"{i + 1}/{len(drug_detail_html)} {row['code']}")

1/6388 A11ABBBBB2281
2/6388 A11APPPPP1610
3/6388 A11A0450A0151
4/6388 A11A0340A0195
5/6388 2021030500011
6/6388 A11A0500A0065
7/6388 A11A0500A0069
8/6388 A11ABBBBB0953
9/6388 A11A0950A0043
10/6388 A11A0970A0064
11/6388 2013103000008
12/6388 A11A0770B0015
13/6388 A11A0950A0068
14/6388 A11A1310A0026
15/6388 A11ABBBBB1168
16/6388 A11A0120A0286
17/6388 A11A2780A0045
18/6388 A11A1270A0036
19/6388 2017081600006
20/6388 2017081600007
21/6388 A11A1660A0057
22/6388 A11A0570A0188
23/6388 A11ABBBBB0607
24/6388 A11A0450A0294
25/6388 A11A0450A0295
26/6388 A11A5060A0010
27/6388 A11A5060A0017
28/6388 A11A0950A0131
29/6388 A11A1110A0058
30/6388 A11A1310A0130
31/6388 A11AFFFFF0079
32/6388 2021012100004
33/6388 A11A1270A0060
34/6388 A11AMMMMM1151
35/6388 2011122300007
36/6388 2012011100004
37/6388 2019121000003
38/6388 2020100700002
39/6388 2020102900010
40/6388 2021111700001
41/6388 A11A1310A0131
42/6388 A11A1310A0137
43/6388 A11A0500A0112
44/6388 A11APPPPP0568
45/6388 A11A0430A0168
46/6388 A11A0740A02

In [38]:
result_drug_df = pd.DataFrame(detail_data)
result_drug_df.to_csv(data_base_path + "result_drug_detail.csv")

In [45]:
result_drug_df['제품명'].value_counts()

제품명
설박신정500mgSulbaxin Tab. 500mg                               119
동화암로디핀베실산염정10mgAmlodipine Tab. 10mg Dong Wha (besylate)     15
동화암로디핀베실산염정5mgDongwha Amlodipine Besylate Tab. 5mg          15
우먼스타이레놀정Women's Tylenol Tab.                                 5
복합파자임이중정Complex Phazyme two-layer Tab.                       4
                                                          ... 
파모타제정10mgFamotase Tab. 10mg                                  1
위엔싹에프정Wienssak F Tab.                                        1
뉴락스에프정Newlax F Tab.                                          1
휴메트린정Humetrine Tab.                                          1
레코미드서방정150mgRecomid SR Tab. 150mg                            1
Name: count, Length: 5698, dtype: int64

In [49]:
result_drug_df = result_drug_df.drop_duplicates(subset = ["제품명"])

In [50]:
result_drug_df['제품명'].value_counts()

제품명
레코미드서방정150mgRecomid SR Tab. 150mg                          1
게루삼정Gelusam Tab.                                           1
휴텍스아스피린장용정100mgHutecs Aspirin Enteric Coated Tab. 100mg    1
가스모틴에스알정Gasmotin SR Tab.                                   1
센스데이정Senseday Tab.                                         1
                                                          ..
자이로릭정Zyroric Tab.                                          1
셉트린정Septrin Tab.                                           1
아로나민골드프리미엄정Aronamin Gold Premium Tab.                      1
아로나민골드정Aronamin Gold Tab.                                  1
마이암부톨제피정400mgMyambutol Tab. 400mg                          1
Name: count, Length: 5698, dtype: int64

In [ ]:
result_drug_df.to_csv(data_base_path + "result_drug_detail.csv", index=False)

In [53]:
result_drug_df

,제품명,성분/함량,첨가물,제형,성상,KPIC/ATC,구분,효능,용법,주의사항
0,게루삼정Gelusam Tab.,"[Calcium Carbonate P.P.T. 침강탄산칼슘 100mg, Dried ...","[스테아르산마그네슘, 카르복시메틸셀룰로오스칼슘]",정제,흰색의 원형정제,"[소화기계질환, 소화성궤양 치료제, 제산제, 흡수성, 소화기계질환, 소화성궤양 치료...","기밀용기, 실온(1~30℃)보관","위산과다, 속쓰림, 위부불쾌감, 위부팽만감, 체함, 구역, 구토, 위통, 신트림.","만 15세 이상 및 성인: 1회 2정, 1일 3회",1. 다음과 같은 사람은 이 약을 복용하지 말 것. 1) 투석요법을 받고 있는 환자...
1,지엘타이밍정GL Timing Tab.,[Caffeine Anhydrous 카페인무수물 50mg],"[경질무수규산, 리보플라빈, 스테아르산마그네슘]",정제,노란색의 팔각형 정제,"[신경계 질환, 기타 신경계용 약물, 기타]","밀폐용기, 실온(1~30℃)보관",졸음,성인 : 무수카페인으로서 1회 100-300㎎을 1일 1-3회 경구 투여한다.,1. 다음과 같은 사람은 이 약을 복용하지 말 것 이 약은 유당(젖당)을 함유하고 ...
2,마이암부톨제피정400mgMyambutol Tab. 400mg,[Ethambutol Hydrochloride 에탐부톨염산염 400mg],"[D-소르비톨액, 모노라우린산소르비탄, 백당]",정제,황색의 원형필름코팅정제,"[감염성질환, 항결핵제, 항균제, 2차약물]","기밀용기, 실온(1 ~ 30℃)보관",1. 유효균종 결핵균 2. 적응증 폐결핵 및 기타 결핵증,1. 초기치료 염산에탐부톨로서 1일 체중 kg당 15 mg을 24시간마다 1회 경구...,1. 다음 환자에는 투여하지 말 것. 1) 이 약 또는 이 약의 구성성분에 과민반응...
3,아로나민골드정Aronamin Gold Tab.,[Ascorbic Acid D.C. 직타용아스코르브산 72.2mg (아스코르브산(으...,"[미결정셀룰로오스, 스테아르산마그네슘, 오파글로스클리어(97W19196)]",정제,1. 내수용 : 이 약은 적색의 필름코팅정이다.2. 수출용 : 이 약은 빨간색의 필...,"[비타민 및 영양제류, 수용성 비타민, 비타민C 제제, 비타민 및 영양제류, 수용성...","기밀용기, [내수용] 실온(1~30℃)보관, [수출용] 25℃ 이하 보관","○ 다음 경우의 비타민 B1, B2, B6, E, C의 보급",성인 1회 1정 1일 2회 복용한다,1) 다음과 같은 사람은 이 약을 복용하지 말 것. (1) 이 약 및 이 약에 포함...
4,아로나민골드프리미엄정Aronamin Gold Premium Tab.,"[Biotin 비오틴 0.2mg, Calcium Pantothenate 판토텐산칼슘...","[미결정셀룰로오스, 스테아르산마그네슘, 오파글로스클리어(97W19196)]",정제,적색의 타원형 필름코팅정,"[비타민 및 영양제류, 수용성 비타민, 비타민B 제제, 비타민B7, 비타민 및 영양...","기밀용기, 실온(1~30℃)보관","○ 다음 경우의 비타민 D, E, B 1 , B 2 , B 6 ,의 보급",8세 이상 및 성인: 1일 1~2회 1회 1정,1. 다음과 같은 사람은 이 약을 복용하지 말 것. 1) 고칼슘혈증(hypercal...
...,...,...,...,...,...,...,...,...,...,...
6383,파모타제정10mgFamotase Tab. 10mg,[Famotidine 파모티딘 10mg],"[미결정셀룰로오스, 산화티탄, 스테아르산마그네슘]",정제,연녹색의 원형 필름코팅정제,"[소화기계질환, 소화성궤양 치료제, H2수용체 길항제]","기밀용기, 실온(1-30℃)보관","위산과다, 속쓰림, 신트림","16세 이상 1회 1정(10mg), 1일 2정(20mg)까지 복용할 수 있다.",1. 다음과 같은 사람은 이 약을 복용하지 말 것 1) 이 약의 성분 및 다른 H2...
6384,위엔싹에프정Wienssak F Tab.,[Famotidine 파모티딘 10mg],"[미결정셀룰로오스, 스테아르산마그네슘, 오파드라이흰색(85F18422)]",정제,녹색의 원형 필름코팅정,"[소화기계질환, 소화성궤양 치료제, H2수용체 길항제]","기밀용기, 실온(1~30℃)보관","위산과다, 속쓰림, 신트림","16세 이상 1회 1정(10mg), 1일 2정(20mg)까지 복용할 수 있다.",1. 다음 환자에는 투여하지 말 것. 1) 이 약의 성분 및 다른 H2 수용체 길항...
6385,뉴락스에프정Newlax F Tab.,"[Bisacodyl 비사코딜 5mg, Sennae Fructus Powder 센나열...","[경질무수규산, 메타아크릴산공중합체 혼합기제(39G575), 스테아르산마그네슘]",장용정,녹색의 원형 장용성 필름코팅정제,"[소화기계질환, 하제, 대장 자극성 하제, 소화기계질환, 하제, 대장 자극성 하제]","밀폐용기, 실온(1~30℃)보관",‑ 변비,"성인 : 1회 1‑2정, 1일 취침시(공복시)에 복용한다.","1. 다음 환자에는 투여하지 말 것. 1) 급성 복부질환(충수염, 장출혈, 궤양성 ..."
6386,휴메트린정Humetrine Tab.,[Methylergometrine Maleate 메틸에르고메트린말레산염 0.125mg],"[농글리세린, 말레산, 미결정셀룰로오스]",정제,노란색의 원형 당의정,"[비뇨생식기계질환, 자궁에 작용하는 약물, 자궁수축제, 맥각 알칼로이드]","차광한 기밀용기, 실온(1-30℃)보관",다음의 경우 출혈의 방지 및 치료:,성인 : 보통 메틸에르고메트린말레산염으로서 1회 0.125 ‑0.25mg을 1일 2...,1. 다음 환자에는 투여하지 말 것. 1) 임부 및 임신하고 있을 가능성이 있는 부...


---

In [48]:
import pandas as pd

# CSV 불러오기
result_df = pd.read_csv(data_base_path + "result_drug_detail.csv")
single_df = pd.read_csv(data_base_path + "single_drug_list_5000.csv")

In [49]:
matches = []

for i, row in result_df.iterrows():
    prod_name = str(row["제품명"])  # 제품명
    match_row = single_df[single_df["name"].notna() & 
                          single_df["name"].apply(lambda x: str(x) in prod_name)]
    
    if not match_row.empty:
        if len(match_row) > 1:
            print(match_row)
        matches.append(match_row.iloc[0])
    else:
        matches.append(pd.Series())

matched_df = pd.DataFrame(matches).reset_index(drop=True)
merged_df = pd.concat([result_df.reset_index(drop=True), matched_df], axis=1)

        C-Code     구분              제품명  복지부분류코드 volume         name
3     K-000112  일반의약품           아로나민골드    316.0    NaN       아로나민골드
4566  K-059707  일반의약품  아로나민골드프리미엄정 120    316.0    NaN  아로나민골드프리미엄정
        C-Code     구분      제품명  복지부분류코드 volume     name
19    K-000438  전문의약품  네오메디코푸정    222.0    NaN  네오메디코푸정
4674  K-005986  전문의약품      코푸정    222.0    NaN      코푸정
        C-Code     구분             제품명  복지부분류코드 volume        name
24    K-000573  일반의약품         게보린 10T    114.0    10T         게보린
4355  K-049847  일반의약품  게보린소프트연질캡슐 12C    114.0    12C  게보린소프트연질캡슐
       C-Code     구분          제품명  복지부분류코드 volume     name
24   K-000573  일반의약품      게보린 10T    114.0    10T      게보린
986  K-052916  일반의약품  게보린쿨다운정 30T    114.0    30T  게보린쿨다운정
        C-Code     구분         제품명  복지부분류코드 volume        name
24    K-000573  일반의약품     게보린 10T    114.0    10T         게보린
4538  K-053013  일반의약품  게보린릴랙스연질캡슐    114.0    NaN  게보린릴랙스연질캡슐
        C-Code     구분                제품명  복지부분류코드 volume   name
35 

In [51]:
merged_df = merged_df.dropna(subset='name')

In [ ]:
merged_df.isna().sum()

제품명            0
성분/함량          0
첨가물            0
제형             0
성상             0
KPIC/ATC       0
구분             0
효능             4
용법             0
주의사항           2
C-Code         0
구분             0
제품명            0
복지부분류코드        0
volume      1199
name           0
dtype: int64

In [58]:
merged_df.shape

(5684, 16)

In [54]:
merged_unique = merged_df.drop_duplicates(subset=["name"], keep="first")

In [57]:
merged_unique.shape

(3604, 16)

In [59]:
import pandas as pd
import re

# ---- 단위 파싱 유틸 ----
UNIT_MAP = {
    # base: mg(질량), mL(부피)
    "mg": ("mass", 1.0),
    "g":  ("mass", 1000.0),
    "mcg":("mass", 0.001),
    "μg": ("mass", 0.001),
    "ug": ("mass", 0.001),
    "ml": ("vol", 1.0),
    "l":  ("vol", 1000.0),
    "iu": ("iu", 1.0),   # 별개 차원 취급
    "%":  ("pct", 1.0),  # 별개 차원 취급
}
VOL_RE = re.compile(r'(?P<value>\d+(?:\.\d+)?)\s*(?P<unit>mg|g|mcg|μg|ug|ml|l|iu|%)', re.I)

def normalize_unit(u: str):
    u = u.lower()
    if u in ("μg", "ug"): u = "mcg"
    return UNIT_MAP.get(u, (None, None))

def parse_volume(text: str):
    """(dim, base_value) 반환. base: mg / mL. 못 찾으면 (None, None)."""
    if not isinstance(text, str): return (None, None)
    m = VOL_RE.search(text)
    if not m: return (None, None)
    val = float(m.group("value"))
    unit = m.group("unit")
    dim, mult = normalize_unit(unit)
    if dim is None or mult is None: return (None, None)
    return dim, val * mult

def rel_distance(v1: float, v2: float):
    if v1 is None or v2 is None: return float("inf")
    if v1 == 0 and v2 == 0: return 0.0
    if v1 == 0 or v2 == 0: return float("inf")
    return abs(v1 - v2) / max(v1, v2)

# ---- 데이터 로드 ----
result_df = pd.read_csv(data_base_path + "result_drug_detail.csv")
single_df = pd.read_csv(data_base_path + "single_drug_list_5000.csv")

# 전처리 (소문자)
single_df["name_lc"] = single_df["name"].astype(str).str.lower().str.strip()

# single_df volume 미리 파싱
single_df[["_dim","_base"]] = single_df["volume"].apply(
    lambda x: pd.Series(parse_volume(x))
)

# ---- 매칭: name 포함 + volume 가장 근접 ----
picked_rows = []
dists = []

for prod_name in result_df["제품명"].astype(str):
    prod_lc = prod_name.lower()
    # 후보: name이 제품명에 포함
    cands = single_df[single_df["name_lc"].apply(lambda nm: isinstance(nm, str) and nm and (nm in prod_lc))]
    if cands.empty:
        picked_rows.append(pd.Series(dtype=object))
        dists.append(None)
        continue

    # 제품명에서 volume 파싱
    prod_dim, prod_base = parse_volume(prod_name)

    # 스코어: (차원 같으면 상대오차, 다르면 inf) → 동점이면 name 길이 긴 것 우선
    scores = []
    for idx, row in cands.iterrows():
        cand_dim, cand_base = row["_dim"], row["_base"]
        dist = float("inf")
        if prod_dim is not None and cand_dim is not None and prod_dim == cand_dim:
            dist = rel_distance(prod_base, cand_base)
        # tie-breaker: 더 구체적인 이름(길이 긴 것) 우선
        name_len = len(row["name_lc"]) if isinstance(row["name_lc"], str) else 0
        scores.append((dist, -name_len, idx))

    scores.sort(key=lambda x: (x[0], x[1]))
    best_idx = scores[0][2]
    picked_rows.append(single_df.loc[best_idx])
    dists.append(scores[0][0])

matched_df = pd.DataFrame(picked_rows).reset_index(drop=True)
matched_df = matched_df.rename(columns={"제품명": "single_제품명"})  # 충돌 방지

# ---- 필요한 컬럼만 이어붙이기 ----
cols_to_add = ["C-Code", "구분", "single_제품명", "복지부분류코드", "volume", "name"]
out_df = pd.concat([result_df.reset_index(drop=True), matched_df.get(cols_to_add)], axis=1)
out_df["name_match_volume_distance"] = dists


In [60]:
out_df['single_제품명'].value_counts()

single_제품명
부스피론정 5MG (명인)    10
콕시브캡슐100mg        10
미녹시딜정 5MG (현대)    10
로프람정 10mg         10
로딘정 200MG(건일)      9
                  ..
파모타제정10mg 20T      1
위엔싹에프정 10T         1
뉴락스에프정 10T         1
휴메트린정0.125mg       1
쓱조인정               1
Name: count, Length: 4157, dtype: int64

In [61]:
out_unique = out_df.drop_duplicates(subset=["single_제품명"], keep="first")

In [69]:
out_unique['single_제품명'].value_counts()

single_제품명
레코미드서방정 150MG(유한)      1
게루삼정 200T              1
지엘타이밍정 12T             1
마이암부톨제피정400mg          1
아로나민골드                 1
                      ..
로프민캡슐(지사제)(비급여) 10C    1
페리나정 25mg 10T          1
구주스피로닥톤정25mg           1
낙센정250mg 10T           1
마그밀정 500mg(삼남)         1
Name: count, Length: 4157, dtype: int64

In [64]:
out_unique.shape

(4158, 17)

In [66]:
out_unique['name'].value_counts()

name
씬지록신정         6
스트라테라캡슐       5
리리카캡슐         5
쎄로켈서방정        5
옥시크로린정        5
             ..
게보린소프트연질캡슐    1
게보린쿨다운정       1
게보린릴랙스연질캡슐    1
싸이메트정         1
가스모틴에스알정      1
Name: count, Length: 3694, dtype: int64

In [67]:
out_unique[out_unique['name'] == "씬지록신정"]

,제품명,성분/함량,첨가물,제형,성상,KPIC/ATC,구분,효능,용법,주의사항,C-Code,구분,single_제품명,복지부분류코드,volume,name,name_match_volume_distance
1232,씬지록신정100μgSynthyroxine Tab. 100μg,['Levothyroxine Sodium Hydrate\u3000레보티록신나트륨수화...,"['메글루민', '스테아르산마그네슘', '옥수수전분']",정제,황색의 타원형 정제,"['호르몬 & 대사관련 질환', '갑상선 질환 치료제', '갑상선 기능 저하증 치료...","차광기밀용기, 실온(1-30℃)보관","갑상선기능저하증, 점액부종, 크레틴병",성인 : 레보치록신나트륨으로서 보통 1일 1회 25‑400㎍ 경구투여한다. 일반적인...,1. 경고 갑상선 기능이 정상인 사람이 체중감소를 위해 갑상선 호르몬제를 단독 또는...,K-042340,전문의약품,씬지록신정 88MCG(다림),243.0,88MCG,씬지록신정,0.12
1233,씬지록신정50μgSynthyroxine Tab. 50㎍,['Levothyroxine Sodium Hydrate\u3000레보티록신나트륨수화...,"['메글루민', '스테아르산마그네슘', '옥수수전분']",정제,흰색의 타원형 정제,"['호르몬 & 대사관련 질환', '갑상선 질환 치료제', '갑상선 기능 저하증 치료...","차광기밀용기,실온25℃이하 보관","갑상선기능저하증, 점액부종, 크레틴병",성인 : 레보치록신나트륨으로서 보통 1일 1회 25 ‑400㎍ 경구투여한다. 일반적...,1. 경고 갑상선 기능이 정상인 사람이 체중감소를 위해 갑상선 호르몬제를 단독 또는...,K-013103,전문의약품,씬지록신정 50MCG (다림),243.0,50MCG,씬지록신정,0.00
1234,씬지록신정150μgSynthyroxine Tab. 150μg,['Levothyroxine Sodium Hydrate\u3000레보티록신나트륨수화...,"['메글루민', '스테아르산마그네슘', '옥수수전분']",정제,회보라색의 타원형 정제,"['호르몬 & 대사관련 질환', '갑상선 질환 치료제', '갑상선 기능 저하증 치료...","차광기밀용기, 25℃ 이하 보관","갑상선기능저하증, 점액부종, 크레틴병",성인 : 레보티록신나트륨으로서 보통 1일 1회 25-400㎍ 경구투여한다. 일반적인...,1. 경고 갑상선 기능이 정상인 사람이 체중감소를 위해 갑상선 호르몬제를 단독 또는...,K-040119,전문의약품,씬지록신정 150MCG(다림),243.0,150MCG,씬지록신정,0.00
1235,씬지록신정75μgSynthyroxine Tab. 75μg,['Levothyroxine Sodium Hydrate\u3000레보티록신나트륨수화...,"['메글루민', '스테아르산마그네슘', '옥수수전분']",정제,녹색의 타원형 정제,"['호르몬 & 대사관련 질환', '갑상선 질환 치료제', '갑상선 기능 저하증 치료...","차광기밀용기, 실온(1~30℃)보관","갑상선기능저하증, 점액부종, 크레틴병",성인 : 레보티록신나트륨으로서 보통 1일 1회 25-400㎍ 경구투여한다. 일반적인...,1. 경고 갑상선 기능이 정상인 사람이 체중감소를 위해 갑상선 호르몬제를 단독 또는...,K-042103,전문의약품,씬지록신정 75MCG(다림),243.0,75MCG,씬지록신정,0.00
1236,씬지록신정125μgSynthyroxine Tab. 125μg,['Levothyroxine Sodium Hydrate\u3000레보티록신나트륨수화...,"['메글루민', '스테아르산마그네슘', '옥수수전분']",정제,청색의 타원형 정제,"['호르몬 & 대사관련 질환', '갑상선 질환 치료제', '갑상선 기능 저하증 치료...","차광기밀용기, 25℃ 이하 보관","갑상선기능저하증, 점액부종, 크레틴병",성인 : 레보티록신나트륨으로서 보통 1일 1회 25-400㎍ 경구투여한다. 일반적인...,1. 경고 갑상선 기능이 정상인 사람이 체중감소를 위해 갑상선 호르몬제를 단독 또는...,K-042262,전문의약품,씬지록신정 125MCG(다림),243.0,125MCG,씬지록신정,0.00
1237,씬지록신정25μgSynthyroxine Tab. 25μg,['Levothyroxine Sodium Hydrate\u3000레보티록신나트륨수화...,"['메글루민', '스테아르산마그네슘', '옥수수전분']",정제,분홍색의 타원형 정제,"['호르몬 & 대사관련 질환', '갑상선 질환 치료제', '갑상선 기능 저하증 치료...","차광기밀용기, 25℃ 이하 보관","갑상선기능저하증, 점액부종, 크레틴병",성인 : 레보티록신나트륨으로서 보통 1일 1회 25-400㎍ 경구투여한다. 일반적인...,1. 경고 갑상선 기능이 정상인 사람이 체중감소를 위해 갑상선 호르몬제를 단독 또는...,K-042269,전문의약품,씬지록신정 25MCG(다림),243.0,25MCG,씬지록신정,0.00


In [70]:
out_unique.to_csv(data_base_path + 'result_unique_drug.csv', index=False)